# Inlämningsuppgift: Del 2 - Adversarial input attack

Philip Wollsén Ervius \
phao21@student.bth.se

Amin Afzali \
moaf@student.bth.se

# Instruktioner

Notebookens underrubriker på nivå 3 (###) är bara ett enkelt exempel på hur er implementation kan delas upp. Ni bör skapa egna rubriker anpassade efter er implementation men lämna alla nivå 1 (#) och 2-rubriker (##) som de är. En viktig del inom data-science är att kunna presentera sina metoder och resultat på ett tydligt sätt. **En ostrukturerad och otydlig notebook kan påverka betygsättningen.**

Notera att t.ex. hyperparameter-tuning och annan optimisering för prestanda inte är nödvändigt i denna uppgift. Så länge modellen fungerar märkvärt bättre än ett "coin flip" så räcker det att ni väljer era hyperparametrar manuellt.

# Uppgiften

## Beskrivning av adversarial input attacker

Börja med att förklara vad denna typ av attacker är och hur de fungerar i rapportens sektion 2.1.

## Implementation av er attack

Beskriv i er rapport vad just er valda attack kallas och i detalj hur den fungerar (sektion 2.2 i rapporten). Beskriv även kort vilka bibliotek ni använder för att implementera den.

## Er kod för attacken

### Bibliotek

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image


#### A pretrained ResNet50 model will be used

In [ ]:
# Load pretrained ResNet-50 model

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

model.eval();

### Förberedelser

In [ ]:
# These two functions allow us to get the names for labels, and vice versa.

url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
urllib.request.urlretrieve(url, "imagenet_classes.txt")
with open("imagenet_classes.txt") as f:
    id_to_label = [line.strip() for line in f.readlines()]

def get_class_name(label: int) -> str:
    """Returns the name of class as string."""
    return id_to_label[label]

def get_label(name: str) -> int:
    """Returns the label corresponding to name."""
    return id_to_label.index(name)


### Egna funktioner

In [ ]:
# Define image transformations

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

unnormalize = transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225], std=[1 / 0.229, 1 / 0.224, 1 / 0.225])

# Function to load and preprocess image
def load_image(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

def classify_image(img) -> int:
    """Classifies one image and returns the prediction label.

    Parameters:
        - img: tensor of preprocessed image.

    Returns:
        - int predicted label for the image."""

    with torch.no_grad():
        output = model(img)
    _, pred = torch.max(output, 1)
    return pred

def display_classifications(input_img_1 = "images/koala.jpeg", input_img_2 = "images/tractor.jpeg"):
    """Classifies and displays two images.

    Default images are koala and tractor."""

    # Plot the images with predictions
    fig, axes = plt.subplots(1, 2, figsize=(13, 6))

    # Display predictions

    if isinstance(input_img_1, str):
        image_1 = load_image(input_img_1)
    else:
        image_1 = input_img_1

    if isinstance(input_img_2, str):
        image_2 = load_image(input_img_2)
    else:
        image_2 = input_img_2

    image_1.requires_grad_(False)
    image_2.requires_grad_(False)


    axes[0].imshow(unnormalize(image_1.squeeze(0)).permute(1, 2, 0).numpy())
    axes[1].imshow(unnormalize(image_2.squeeze(0)).permute(1, 2, 0).numpy())

    koala_pred = classify_image(image_1)
    tractor_pred = classify_image(image_2)

    axes[0].set_title(f'Predicted: {get_class_name(koala_pred.item())}')
    axes[1].set_title(f'Predicted: {get_class_name(tractor_pred.item())}')

    axes[0].axis('off')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()


display_classifications()


In [ ]:
def train(img, label):
    """Trains the model on a single image.
    
    Parameters:
        - img: tensor of preprocessed image.
        - label: int target label for the image.
    """

    # Define loss function and optimizer
    criterion = CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)

    # Set the model to training mode
    model.train()
    running_loss = 0.0

    # Prepare the input and label
    labels = torch.tensor([label])

    optimizer.zero_grad()
    outputs = model(img)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    print(f'Loss: {running_loss:.3f}')
    # print('Finished Training')


In [ ]:
def calculate_gradient(img, label):
    """Calculates the gradient of the loss with respect to the input image.
    
    Parameters:
        - img: tensor of preprocessed image.
        - label: int target label for the image.
        
    Returns:
        - gradients: tensor of gradients with respect to the input image.
    """

    # Ensure the input tensor requires gradients
    img.requires_grad_(True)
    
    criterion = CrossEntropyLoss()
    labels = torch.tensor([label])

    outputs = model(img)
    loss = criterion(outputs, labels)
    loss.backward()

    gradients = img.grad
    return gradients


In [ ]:

def create_attack_image(img, label, delta = 0.001):

    output = img.detach().clone()
    output.requires_grad_(True)
    max_iter = 100

    for i in range(max_iter):
        print(i)

        if classify_image(output) == label:
            print(f"Attack image successful with noice strength {i * delta}")
            break

        gradient = calculate_gradient(output, label)
        output = output + gradient * delta * i

    return output


In [ ]:
# Samla era egna funktioner relaterade till attacken här
# Funktionerna ska kort beskrivas med docstrings och kommentarer så att man förstår syftet och implementationen
# Blir det många bör ni bryta ut dessa i en separat fil

### Attack

Beskriv attackprocessen kortfattat här

In [ ]:
img = torch.randn(1, 3, 224, 224, requires_grad=True)
label = get_label("tractor")

output = create_attack_image(img, label, 0.0001)

display_classifications(img, output)

# display_classifications(img, output)


In [ ]:
# Attackutförande

# Load the two images (koala & tractor)
koala_image = load_image("images/koala.jpeg")
tractor_image = load_image("images/tractor.jpeg")

while (classify_image(tractor_image) != classify_image(koala_image)):
    train(koala_image, get_label("tractor"))


### Attackresultat

Summera och förklara era attackresultat i flytande text här

In [ ]:
# Presentera attackens resultat m.h.a. kod här
display_classifications()

## Skyddsåtgärder

Beskriv (i rapportens sektion 2.3) vilka säkerhetsåtgärd(er) ni valt som lämpliga för att skydda mot er attack. Motivera varför ni valt just de(n) metod(erna).

## Implementation av skyddsåtgärder (frivilligt endast för A eller B)

Nedan fyller ni i er kod som implementerar skyddsmekanismen mot er attackmetod. Skriv även en summering av vad skyddsresultatet blev när ni implementerade metoden (beskriv detta i rapportens del 2.4).

### Egna funktioner

In [ ]:
# Samla era egna funktioner relaterade till säkerhetsåtgärden här
# Funktionerna ska kort beskrivas med docstrings och kommentarer så att man förstår syftet och implementationen
# Blir det många bör ni bryta ut dessa i en separat fil

### Förberedelser

In [ ]:
# E.g. skapande av ny modell, etc.

### Skydd

Beskriv säkerhetsåtgärden kortfattat här

In [ ]:
# Implementation av skyddsåtgärd

### Skyddsresultat

Bevisa och förklara era skyddsresultat i flytande text här

In [ ]:
# Presentera skyddåtgärdens resultat m.h.a. kod här

# Referenser

Lista era referenser här. E.g. var ni hittat information om attacken, skyddsåtgärder eller bibliotekets dokumentation/repository.